In [5]:

import sys, os

import importlib, time
import traceback
import numpy as np
import time

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader

from humor.utils.config_new import ConfigParser
from humor.utils.logging import Logger, class_name_to_file_name, mkdir, cp_files
from humor.utils.torch import get_device, save_state, load_state
from humor.utils.stats import StatTracker
from humor.models.model_utils import step

NUM_WORKERS = 0
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
config_file = "configs/train_testmodel.yaml"
config_parser_yaml = ConfigParser(config_file)
args_obj, _ = config_parser_yaml.parse('train')

Using default: {'beta1', 'ckpt', 'beta2', 'decay', 'load_optim', 'eps', 'use_adam'}
Using default: {'detach_sched_samp', 'model_use_smpl_joint_inputs', 'output_delta'}
Using default: {'splits_path', 'frames_out_step_size', 'data_noise_std'}
Using default: {'kl_loss_cycle_len', 'smpl_vert_consistency_loss'}


In [7]:
# See config
dict_attr = ['base_dict', 'model_dict', 'dataset_dict', 'loss_dict']
for attr in dict_attr:
    print(f"{attr}: {getattr(args_obj, attr)}")

base_dict: {'dataset': 'AmassDiscreteDataset', 'model': 'TestModel', 'loss': 'HumorLoss', 'out': './out/testmodel_train_test', 'ckpt': None, 'gpu': 0, 'batch_size': 200, 'print_every': 10, 'epochs': 200, 'val_every': 2, 'save_every': 25, 'lr': 0.0001, 'beta1': 0.9, 'beta2': 0.999, 'eps': 1e-08, 'sched_milestones': [50, 80, 140], 'sched_decay': 0.5, 'decay': 0.0, 'load_optim': True, 'use_adam': False, 'sched_samp_start': 2, 'sched_samp_end': 10}
model_dict: {'out_rot_rep': 'aa', 'in_rot_rep': 'mat', 'latent_size': 48, 'steps_in': 1, 'conditional_prior': False, 'output_delta': True, 'model_data_config': 'smpl+joints+contacts', 'detach_sched_samp': True, 'model_use_smpl_joint_inputs': False}
dataset_dict: {'data_paths': ['../datasets/AMASS/amass_processed'], 'split_by': 'sequence', 'splits_path': None, 'sample_num_frames': 10, 'data_rot_rep': 'mat', 'step_frames_in': 1, 'step_frames_out': 1, 'frames_out_step_size': 1, 'data_return_config': 'smpl+joints+contacts', 'data_noise_std': 0.0}
lo

In [8]:
args = args_obj.base
args.out = os.path.join(args.out, time.strftime('%Y%m%d_%H%M%S'))
mkdir(args.out)
train_log_path = os.path.join(args.out, 'train.log')
Logger.init(train_log_path)
   
# save arguments used
Logger.log('Base args: ' + str(args))
Logger.log('Model args: ' + str(args_obj.model))
Logger.log('Dataset args: ' + str(args_obj.dataset))
Logger.log('Loss args: ' + str(args_obj.loss))

train_scripts_path = os.path.join(args.out, 'train_scripts')
mkdir(train_scripts_path)
pkg_root = "humor"
dataset_file = class_name_to_file_name(args.dataset)
dataset_file_path = os.path.join(pkg_root, 'datasets/' + dataset_file + '.py')
model_file = class_name_to_file_name(args.model)
model_file_path = os.path.join(pkg_root, 'models/' + model_file + '.py')
loss_file = class_name_to_file_name(args.loss)
train_file_path = "train.ipynb"
cp_files(train_scripts_path, [train_file_path, model_file_path, dataset_file_path, config_file])

Base args: namespace(dataset='AmassDiscreteDataset', model='TestModel', loss='HumorLoss', out='./out/testmodel_train_test\\20250424_145116', ckpt=None, gpu=0, batch_size=200, print_every=10, epochs=200, val_every=2, save_every=25, lr=0.0001, beta1=0.9, beta2=0.999, eps=1e-08, sched_milestones=[50, 80, 140], sched_decay=0.5, decay=0.0, load_optim=True, use_adam=False, sched_samp_start=2, sched_samp_end=10)
Model args: namespace(out_rot_rep='aa', in_rot_rep='mat', latent_size=48, steps_in=1, conditional_prior=False, output_delta=True, model_data_config='smpl+joints+contacts', detach_sched_samp=True, model_use_smpl_joint_inputs=False)
Dataset args: namespace(data_paths=['../datasets/AMASS/amass_processed'], split_by='sequence', splits_path=None, sample_num_frames=10, data_rot_rep='mat', step_frames_in=1, step_frames_out=1, frames_out_step_size=1, data_return_config='smpl+joints+contacts', data_noise_std=0.0)
Loss args: namespace(kl_loss=0.0004, kl_loss_anneal_start=0, kl_loss_anneal_end=5

In [9]:
model_class = importlib.import_module('models.' + model_file)
print('Model class: ', model_class)
Model = getattr(model_class, args.model)
model = Model(**args_obj.model_dict,
                model_smpl_batch_size=args.batch_size) # assumes model is HumorModel

Model class:  <module 'models.test_model' from 'E:\\workspace\\Motion\\humor_local\\humor\\utils\\..\\models\\test_model.py'>


In [10]:
  # load loss class and instantiate
loss_class = importlib.import_module('losses.' + loss_file)
print('Loss class: ', loss_class)
Loss = getattr(loss_class, args.loss)
loss_func = Loss(**args_obj.loss_dict,
                    smpl_batch_size=args.batch_size*args_obj.dataset.sample_num_frames) # assumes loss is HumorLoss

Loss class:  <module 'losses.humor_loss' from 'E:\\workspace\\Motion\\humor_local\\humor\\utils\\..\\losses\\humor_loss.py'>


In [11]:

device = get_device(args.gpu)
model.to(device)
loss_func.to(device)

print(model)

# count params in model
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
Logger.log('Num model params: ' + str(params))

# freeze params in loss
for param in loss_func.parameters():
    param.requires_grad = False

Using detected GPU...
TestModel(
  (mlp_test): Sequential(
    (0): Linear(in_features=339, out_features=512, bias=True)
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=512, out_features=1024, bias=True)
    (4): ELU(alpha=1.0)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ELU(alpha=1.0)
    (8): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=512, out_features=96, bias=True)
  )
  (decode_test): Sequential(
    (0): Linear(in_features=48, out_features=512, bias=True)
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=512, out_features=1024, bias=True)
    (4): ELU(alpha=1.0)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, af

In [12]:
# optimizer
betas = (args.beta1, args.beta2)
if args.use_adam:
    optimizer = optim.Adam(model.parameters(),
                            lr=args.lr,
                            betas=betas,
                            eps=args.eps,
                            weight_decay=args.decay)
else:
    optimizer = optim.Adamax(model.parameters(),
                            lr=args.lr,
                            betas=betas,
                            eps=args.eps,
                            weight_decay=args.decay)

# load in pretrained weights/optimizer state if given
start_epoch = 0
min_val_loss = min_train_loss = float('inf')
if args.ckpt is not None:
    load_optim = optimizer if args.load_optim else None
    start_epoch, min_val_loss, min_train_loss = load_state(args.ckpt, model, optimizer=load_optim, map_location=device, ignore_keys=model.ignore_keys)
    start_epoch += 1
    Logger.log('Resuming from saved checkpoint at epoch idx %d with min val loss %.6f...' % (start_epoch, min_val_loss))
    if not args.load_optim:
        Logger.log('Not loading optimizer state as desired...')
        Logger.log('WARNING: Also resetting min_val_loss and epoch count!')
        min_val_loss = float('inf')
        start_epoch = 0

# initialize LR scheduler
scheduler = MultiStepLR(optimizer, milestones=args.sched_milestones, gamma=args.sched_decay)

# intialize schedule sampling if desired
use_sched_samp = False
if args.sched_samp_start is not None and args.sched_samp_end is not None:
    if args.sched_samp_start >= 0 and args.sched_samp_end >= args.sched_samp_start:
        Logger.log('Using scheduled sampling starting at epoch %d and ending at epoch %d!' % (args.sched_samp_start, args.sched_samp_end))
        use_sched_samp = True
    else:
        Logger.log('Could not use scheduled sampling with given start and end!')

Using scheduled sampling starting at epoch 2 and ending at epoch 10!


In [13]:
# load dataset class and instantiate training and validation set
Dataset = getattr(importlib.import_module('datasets.' + dataset_file), args.dataset)
print('Dataset class: ', Dataset)
train_dataset = Dataset(split='train', **args_obj.dataset_dict)
val_dataset = Dataset(split='val', **args_obj.dataset_dict)
# create loaders
train_loader = DataLoader(train_dataset, 
                            batch_size=args.batch_size,
                            shuffle=True,
                            num_workers=NUM_WORKERS,
                            pin_memory=True,
                            worker_init_fn=lambda _: np.random.seed()) # get around pytorch RNG seed bug
val_loader = DataLoader(val_dataset, 
                        batch_size=args.batch_size,
                        shuffle=False, 
                        num_workers=NUM_WORKERS,
                        pin_memory=True,
                        worker_init_fn=lambda _: np.random.seed())

Dataset class:  <class 'datasets.amass_discrete_dataset.AmassDiscreteDataset'>
Loading data from../datasets/AMASS/amass_processed
Logger must be initialized before logging!
This split contains 286 sequences (that meet the duration criteria).
Logger must be initialized before logging!
The dataset contains 18411 sub-sequences in total.
Logger must be initialized before logging!
Loading data from../datasets/AMASS/amass_processed
Logger must be initialized before logging!
This split contains 24 sequences (that meet the duration criteria).
Logger must be initialized before logging!
The dataset contains 739 sub-sequences in total.
Logger must be initialized before logging!


In [14]:
# stats tracker
tensorboard_path = os.path.join(args.out, 'train_tensorboard')
mkdir(tensorboard_path)
stat_tracker = StatTracker(tensorboard_path, train_log_path)

# checkpoints saving
ckpts_path = os.path.join(args.out, 'checkpoints')
mkdir(ckpts_path)

if use_sched_samp:
    train_dataset.return_global = True
    val_dataset.return_global = True

In [15]:
# main training loop
train_start_t = time.time()
for epoch in range(start_epoch, args.epochs):

    model.train()

    # train
    stat_tracker.reset()
    batch_start_t = None
    reset_loss_track = train_dataset.pre_batch(epoch=epoch)
    # see which phase we're in 
    sched_samp_gt_p = 1.0 # supervised
    if use_sched_samp:
        if epoch >= args.sched_samp_start and epoch < args.sched_samp_end:
            frac = (epoch - args.sched_samp_start) / (args.sched_samp_end - args.sched_samp_start)
            sched_samp_gt_p = 1.0*(1.0 - frac)
        elif epoch >= args.sched_samp_end:
            # autoregressive
            sched_samp_gt_p = 0.0
        Logger.log('Scheduled sampling current use_gt_p = %f' % (sched_samp_gt_p))

        if epoch == args.sched_samp_end:
            # the loss will naturally go up when using own rollouts
            reset_loss_track = True

        if args_obj.loss_dict['kl_loss_cycle_len'] > 0:
            # if we're cycling, only want to save results when using full ELBO
            if (epoch % args_obj.loss_dict['kl_loss_cycle_len']) > (args_obj.loss_dict['kl_loss_cycle_len'] // 2):
                # have reached second half of a cycle
                reset_loss_track = True

    if reset_loss_track:
        Logger.log('Resetting min_val_loss and min_train_loss')
        min_val_loss = min_train_loss = float('inf')

    for i, data in enumerate(train_loader):
        batch_start_t = time.time()

        try:
            # zero the gradients
            optimizer.zero_grad()
            # run model
            loss, stats_dict = step(model, loss_func, data, train_dataset, device, epoch, mode='train', use_gt_p=sched_samp_gt_p)
            if torch.isnan(loss).item():
                Logger.log('WARNING: NaN loss. Skipping to next data...')
                torch.cuda.empty_cache()
                continue
            # backprop and step
            loss.backward()
            # check gradients
            parameters = [p for p in model.parameters() if p.grad is not None]
            total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), 2.0).to(device) for p in parameters]), 2.0)
            if torch.isnan(total_norm):
                Logger.log('WARNING: NaN gradients. Skipping to next data...')
                torch.cuda.empty_cache()
                continue
            optimizer.step()
        except (RuntimeError, AssertionError) as e:
            if epoch > 0:
                # to catch bad dynamics, but keep training
                Logger.log('WARNING: caught an exception during forward or backward pass. Skipping to next data...')
                Logger.log(e)
                traceback.print_exc()
                reset_loss_track = train_dataset.pre_batch(epoch=epoch)
                if reset_loss_track:
                    Logger.log('Resetting min_val_loss and min_train_loss')
                    min_val_loss = min_train_loss = float('inf')
                continue
            else:
                raise e

        # collect stats
        batch_elapsed_t = time.time() - batch_start_t
        total_elapsed_t = time.time() - train_start_t
        stats_dict['loss'] = loss
        for param_group in optimizer.param_groups:
            stats_dict['lr'] = torch.Tensor([param_group['lr']])[0]
        stats_dict['time_per_batch'] = torch.Tensor([batch_elapsed_t])[0]

        last_batch = (i==(len(train_loader)-1))
        stat_tracker.update(stats_dict, tag='train', save_tf=last_batch)
        if i % args.print_every == 0:
            stat_tracker.print(i, len(train_loader),
                            epoch, args.epochs,
                            total_elapsed_time=total_elapsed_t,
                            tag='train')

        reset_loss_track = train_dataset.pre_batch(epoch=epoch)
        if reset_loss_track:
            Logger.log('Resetting min_val_loss and min_train_loss')
            min_val_loss = min_train_loss = float('inf')

    # save if desired
    if epoch % args.save_every == 0:
        Logger.log('Saving checkpoint...')
        save_file = os.path.join(ckpts_path, 'epoch_%08d_model.pth' % (epoch))
        save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    # check if it's the best train model so far
    mean_train_loss = stat_tracker.meter_dict['train/loss'].avg
    if mean_train_loss < min_train_loss:
        min_train_loss = mean_train_loss
        Logger.log('Best train loss so far! Saving checkpoint...')
        save_file = os.path.join(ckpts_path, 'best_train_model.pth')
        save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    # validate
    if epoch % args.val_every == 0:
        with torch.no_grad():
            # run on validation data
            model.eval()

            stat_tracker.reset()
            for i, data in enumerate(val_loader):
                # print(i)
                batch_start_t = time.time()
                # run model
                loss, stats_dict = step(model, loss_func, data, val_dataset, device, epoch, mode='test', use_gt_p=sched_samp_gt_p)

                if torch.isnan(loss):
                    Logger.log('WARNING: NaN loss on VALIDATION. Skipping to next data...')
                    continue

                # collect stats
                batch_elapsed_t = time.time() - batch_start_t
                total_elapsed_t = time.time() - train_start_t
                stats_dict['loss'] = loss
                stats_dict['time_per_batch'] = torch.Tensor([batch_elapsed_t])[0]

                stat_tracker.update(stats_dict, tag='val', save_tf=(i==(len(val_loader)-1)), increment_step=False)

                if i % args.print_every == 0:
                    stat_tracker.print(i, len(val_loader),
                                    epoch, args.epochs,
                                    total_elapsed_time=total_elapsed_t,
                                    tag='val')

            # check if it's the best model so far
            mean_val_loss = stat_tracker.meter_dict['val/loss'].avg
            if mean_val_loss < min_val_loss:
                min_val_loss = mean_val_loss
                Logger.log('Best val loss so far! Saving checkpoint...')
                save_file = os.path.join(ckpts_path, 'best_model.pth')
                save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    scheduler.step()

    torch.cuda.empty_cache()

Logger.log('Finished!')

Scheduled sampling current use_gt_p = 1.000000
[>--------------------------------------------------] train epoch - 1/200 | batch - 1/93
1.088 s per batch | 2 s elapsed | 23 d 23 h 2 m 4 s ETA
train/kl_loss : 13.10291
train/kl_anneal_weight : 0.00000
train/kl_weighted_loss : 0.00000
train/trans_loss : 0.32664
train/trans_vel_loss : 0.39371
train/root_orient_loss : 0.16943
train/root_orient_vel_loss : 0.90141
train/pose_body_loss : 0.18343
train/joints_loss : 0.32518
train/joints_vel_loss : 0.44647
train/contacts_loss : 0.73437
train/contacts_acc : 0.49772
train/contacts_pos_acc : 0.49615
train/contacts_neg_acc : 0.49838
train/contacts_vel_loss : 2.22637
train/smpl_joint_loss : 0.39690
train/smpl_mesh_loss : 0.43292
train/smpl_joint_consistency_loss : 0.72709
train/reconstr_weighted_loss : 4.33280
train/loss : 4.33280
train/lr : 0.00010
train/time_per_batch : 1.08832
[=====>---------------------------------------------] train epoch - 1/200 | batch - 11/93
0.497 s per batch | 12 s elapsed

E:\workspace\Motion\humor_local\humor\utils\..\utils\transforms.py:26: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Cross.cpp:66.)
  world2aligned_axis = torch.cross(body_right, x_axis.expand_as(body_right))


[>--------------------------------------------------] train epoch - 4/200 | batch - 1/93
0.557 s per batch | 6 m 55 s elapsed | 7 h 34 m 11 s ETA
train/kl_loss : 67.69653
train/kl_anneal_weight : 0.06000
train/kl_weighted_loss : 0.00162
train/trans_loss : 0.00342
train/trans_vel_loss : 0.02055
train/root_orient_loss : 0.00406
train/root_orient_vel_loss : 0.34778
train/pose_body_loss : 0.01127
train/joints_loss : 0.01231
train/joints_vel_loss : 0.09822
train/contacts_loss : 0.69181
train/contacts_acc : 0.54494
train/contacts_pos_acc : 0.75234
train/contacts_neg_acc : 0.46012
train/contacts_vel_loss : 0.67227
train/smpl_joint_loss : 0.00605
train/smpl_mesh_loss : 0.00759
train/smpl_joint_consistency_loss : 0.01722
train/reconstr_weighted_loss : 0.54210
train/loss : 0.54373
train/lr : 0.00010
train/time_per_batch : 0.55682
[=====>---------------------------------------------] train epoch - 4/200 | batch - 11/93
0.485 s per batch | 7 m 5 s elapsed | 7 h 29 m 19 s ETA
train/kl_loss : 68.158

KeyboardInterrupt: 

In [ ]:


def train(args_obj, config_file):

    # # set up output
    # args = args_obj.base
    # mkdir(args.out)

    # # create logging system
    # train_log_path = os.path.join(args.out, 'train.log')
    # Logger.init(train_log_path)

    # # save arguments used
    # Logger.log('Base args: ' + str(args))
    # Logger.log('Model args: ' + str(args_obj.model))
    # Logger.log('Dataset args: ' + str(args_obj.dataset))
    # Logger.log('Loss args: ' + str(args_obj.loss))

    # # save training script/model/dataset used
    # train_scripts_path = os.path.join(args.out, 'train_scripts')
    # mkdir(train_scripts_path)
    # pkg_root = os.path.join(cur_file_path, '..')
    # dataset_file = class_name_to_file_name(args.dataset)
    # dataset_file_path = os.path.join(pkg_root, 'datasets/' + dataset_file + '.py')
    # model_file = class_name_to_file_name(args.model)
    # loss_file = class_name_to_file_name(args.loss)
    # model_file_path = os.path.join(pkg_root, 'models/' + model_file + '.py')
    # train_file_path = os.path.join(pkg_root, 'train/train_humor.py')
    # cp_files(train_scripts_path, [train_file_path, model_file_path, dataset_file_path, config_file])

    # # load model class and instantiate
    # model_class = importlib.import_module('models.' + model_file)
    # Model = getattr(model_class, args.model)
    # model = Model(**args_obj.model_dict,
    #                 model_smpl_batch_size=args.batch_size) # assumes model is HumorModel

    # load loss class and instantiate
    # loss_class = importlib.import_module('losses.' + loss_file)
    # Loss = getattr(loss_class, args.loss)
    # loss_func = Loss(**args_obj.loss_dict,
    #                  smpl_batch_size=args.batch_size*args_obj.dataset.sample_num_frames) # assumes loss is HumorLoss

    # device = get_device(args.gpu)
    # model.to(device)
    # loss_func.to(device)

    # print(model)

    # # count params in model
    # model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    # params = sum([np.prod(p.size()) for p in model_parameters])
    # Logger.log('Num model params: ' + str(params))

    # # freeze params in loss
    # for param in loss_func.parameters():
    #     param.requires_grad = False

    # # optimizer
    # betas = (args.beta1, args.beta2)
    # if args.use_adam:
    #     optimizer = optim.Adam(model.parameters(),
    #                             lr=args.lr,
    #                             betas=betas,
    #                             eps=args.eps,
    #                             weight_decay=args.decay)
    # else:
    #     optimizer = optim.Adamax(model.parameters(),
    #                             lr=args.lr,
    #                             betas=betas,
    #                             eps=args.eps,
    #                             weight_decay=args.decay)

    # # load in pretrained weights/optimizer state if given
    # start_epoch = 0
    # min_val_loss = min_train_loss = float('inf')
    # if args.ckpt is not None:
    #     load_optim = optimizer if args.load_optim else None
    #     start_epoch, min_val_loss, min_train_loss = load_state(args.ckpt, model, optimizer=load_optim, map_location=device, ignore_keys=model.ignore_keys)
    #     start_epoch += 1
    #     Logger.log('Resuming from saved checkpoint at epoch idx %d with min val loss %.6f...' % (start_epoch, min_val_loss))
    #     if not args.load_optim:
    #         Logger.log('Not loading optimizer state as desired...')
    #         Logger.log('WARNING: Also resetting min_val_loss and epoch count!')
    #         min_val_loss = float('inf')
    #         start_epoch = 0

    # # initialize LR scheduler
    # scheduler = MultiStepLR(optimizer, milestones=args.sched_milestones, gamma=args.sched_decay)

    # # intialize schedule sampling if desired
    # use_sched_samp = False
    # if args.sched_samp_start is not None and args.sched_samp_end is not None:
    #     if args.sched_samp_start >= 0 and args.sched_samp_end >= args.sched_samp_start:
    #         Logger.log('Using scheduled sampling starting at epoch %d and ending at epoch %d!' % (args.sched_samp_start, args.sched_samp_end))
    #         use_sched_samp = True
    #     else:
    #         Logger.log('Could not use scheduled sampling with given start and end!')

    # load dataset class and instantiate training and validation set
    # Dataset = getattr(importlib.import_module('datasets.' + dataset_file), args.dataset)
    # train_dataset = Dataset(split='train', **args_obj.dataset_dict)
    # val_dataset = Dataset(split='val', **args_obj.dataset_dict)
    # # create loaders
    # train_loader = DataLoader(train_dataset, 
    #                           batch_size=args.batch_size,
    #                           shuffle=True,
    #                           num_workers=NUM_WORKERS,
    #                           pin_memory=True,
    #                           worker_init_fn=lambda _: np.random.seed()) # get around pytorch RNG seed bug
    # val_loader = DataLoader(val_dataset, 
    #                         batch_size=args.batch_size,
    #                         shuffle=False, 
    #                         num_workers=NUM_WORKERS,
    #                         pin_memory=True,
    #                         worker_init_fn=lambda _: np.random.seed())

    # stats tracker
    # tensorboard_path = os.path.join(args.out, 'train_tensorboard')
    # mkdir(tensorboard_path)
    # stat_tracker = StatTracker(tensorboard_path)

    # # checkpoints saving
    # ckpts_path = os.path.join(args.out, 'checkpoints')
    # mkdir(ckpts_path)

    # if use_sched_samp:
    #     train_dataset.return_global = True
    #     val_dataset.return_global = True

    # # main training loop
    # train_start_t = time.time()
    # for epoch in range(start_epoch, args.epochs):

    #     model.train()

    #     # train
    #     stat_tracker.reset()
    #     batch_start_t = None
    #     reset_loss_track = train_dataset.pre_batch(epoch=epoch)
    #     # see which phase we're in 
    #     sched_samp_gt_p = 1.0 # supervised
    #     if use_sched_samp:
    #         if epoch >= args.sched_samp_start and epoch < args.sched_samp_end:
    #             frac = (epoch - args.sched_samp_start) / (args.sched_samp_end - args.sched_samp_start)
    #             sched_samp_gt_p = 1.0*(1.0 - frac)
    #         elif epoch >= args.sched_samp_end:
    #             # autoregressive
    #             sched_samp_gt_p = 0.0
    #         Logger.log('Scheduled sampling current use_gt_p = %f' % (sched_samp_gt_p))

    #         if epoch == args.sched_samp_end:
    #             # the loss will naturally go up when using own rollouts
    #             reset_loss_track = True

    #         if args_obj.loss_dict['kl_loss_cycle_len'] > 0:
    #             # if we're cycling, only want to save results when using full ELBO
    #             if (epoch % args_obj.loss_dict['kl_loss_cycle_len']) > (args_obj.loss_dict['kl_loss_cycle_len'] // 2):
    #                 # have reached second half of a cycle
    #                 reset_loss_track = True

    #     if reset_loss_track:
    #         Logger.log('Resetting min_val_loss and min_train_loss')
    #         min_val_loss = min_train_loss = float('inf')

    #     for i, data in enumerate(train_loader):
    #         batch_start_t = time.time()

    #         try:
    #             # zero the gradients
    #             optimizer.zero_grad()
    #             # run model
    #             loss, stats_dict = model_class.step(model, loss_func, data, train_dataset, device, epoch, mode='train', use_gt_p=sched_samp_gt_p)
    #             if torch.isnan(loss).item():
    #                 Logger.log('WARNING: NaN loss. Skipping to next data...')
    #                 torch.cuda.empty_cache()
    #                 continue
    #             # backprop and step
    #             loss.backward()
    #             # check gradients
    #             parameters = [p for p in model.parameters() if p.grad is not None]
    #             total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), 2.0).to(device) for p in parameters]), 2.0)
    #             if torch.isnan(total_norm):
    #                 Logger.log('WARNING: NaN gradients. Skipping to next data...')
    #                 torch.cuda.empty_cache()
    #                 continue
    #             optimizer.step()
    #         except (RuntimeError, AssertionError) as e:
    #             if epoch > 0:
    #                 # to catch bad dynamics, but keep training
    #                 Logger.log('WARNING: caught an exception during forward or backward pass. Skipping to next data...')
    #                 Logger.log(e)
    #                 traceback.print_exc()
    #                 reset_loss_track = train_dataset.pre_batch(epoch=epoch)
    #                 if reset_loss_track:
    #                     Logger.log('Resetting min_val_loss and min_train_loss')
    #                     min_val_loss = min_train_loss = float('inf')
    #                 continue
    #             else:
    #                 raise e

    #         # collect stats
    #         batch_elapsed_t = time.time() - batch_start_t
    #         total_elapsed_t = time.time() - train_start_t
    #         stats_dict['loss'] = loss
    #         for param_group in optimizer.param_groups:
    #             stats_dict['lr'] = torch.Tensor([param_group['lr']])[0]
    #         stats_dict['time_per_batch'] = torch.Tensor([batch_elapsed_t])[0]

    #         last_batch = (i==(len(train_loader)-1))
    #         stat_tracker.update(stats_dict, tag='train', save_tf=last_batch)
    #         if i % args.print_every == 0:
    #             stat_tracker.print(i, len(train_loader),
    #                             epoch, args.epochs,
    #                             total_elapsed_time=total_elapsed_t,
    #                             tag='train')

    #         reset_loss_track = train_dataset.pre_batch(epoch=epoch)
    #         if reset_loss_track:
    #             Logger.log('Resetting min_val_loss and min_train_loss')
    #             min_val_loss = min_train_loss = float('inf')

    #     # save if desired
    #     if epoch % args.save_every == 0:
    #         Logger.log('Saving checkpoint...')
    #         save_file = os.path.join(ckpts_path, 'epoch_%08d_model.pth' % (epoch))
    #         save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    #     # check if it's the best train model so far
    #     mean_train_loss = stat_tracker.meter_dict['train/loss'].avg
    #     if mean_train_loss < min_train_loss:
    #         min_train_loss = mean_train_loss
    #         Logger.log('Best train loss so far! Saving checkpoint...')
    #         save_file = os.path.join(ckpts_path, 'best_train_model.pth')
    #         save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    #     # validate
    #     if epoch % args.val_every == 0:
    #         with torch.no_grad():
    #             # run on validation data
    #             model.eval()

    #             stat_tracker.reset()
    #             for i, data in enumerate(val_loader):
    #                 # print(i)
    #                 batch_start_t = time.time()
    #                 # run model
    #                 loss, stats_dict = model_class.step(model, loss_func, data, val_dataset, device, epoch, mode='test', use_gt_p=sched_samp_gt_p)

    #                 if torch.isnan(loss):
    #                     Logger.log('WARNING: NaN loss on VALIDATION. Skipping to next data...')
    #                     continue

    #                 # collect stats
    #                 batch_elapsed_t = time.time() - batch_start_t
    #                 total_elapsed_t = time.time() - train_start_t
    #                 stats_dict['loss'] = loss
    #                 stats_dict['time_per_batch'] = torch.Tensor([batch_elapsed_t])[0]

    #                 stat_tracker.update(stats_dict, tag='val', save_tf=(i==(len(val_loader)-1)), increment_step=False)

    #                 if i % args.print_every == 0:
    #                     stat_tracker.print(i, len(val_loader),
    #                                     epoch, args.epochs,
    #                                     total_elapsed_time=total_elapsed_t,
    #                                     tag='val')

    #             # check if it's the best model so far
    #             mean_val_loss = stat_tracker.meter_dict['val/loss'].avg
    #             if mean_val_loss < min_val_loss:
    #                 min_val_loss = mean_val_loss
    #                 Logger.log('Best val loss so far! Saving checkpoint...')
    #                 save_file = os.path.join(ckpts_path, 'best_model.pth')
    #                 save_state(save_file, model, optimizer, cur_epoch=epoch, min_val_loss=min_val_loss, min_train_loss=min_train_loss, ignore_keys=model.ignore_keys)

    #     scheduler.step()

    #     torch.cuda.empty_cache()

    # Logger.log('Finished!')
    print('Finished!')